<a href="https://colab.research.google.com/github/YooNayoung/ESAA/blob/main/%EC%88%98%EC%83%81%EC%9E%91_%EB%A6%AC%EB%B7%B0_1%ED%9A%8C%EC%B0%A8(%EB%A1%9C%EA%B7%B8_%EB%B6%84%EC%84%9D%EC%9D%84_%ED%86%B5%ED%95%9C_%EB%B3%B4%EC%95%88_%EC%9C%84%ED%97%98%EB%8F%84_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **로그 분석을 통한 보안 위험도 예측 AI 경진대회**

# Data & Libaray IMPORT

1. train.csv
- id : 데이터 식별자
- level : 보안 위험 등급(0~6)
- full_log : 학습 데이터 전체 로그


2. test.csv
- id : 데이터 식별자
- full_log : 테스트 데이터 전체 로그
- train.csv에 존재하지 않는 level 존재(0~7)


3. sample_submission.csv
- id : 데이터 식별자
- level : 예측 level 작성


4. validation_sample.csv
- full_log : 7등급 보안 로그 샘플
- 학습 데이터로 활용 금지
- 학습 완료후 추론과정 검증데이터로만 활용 가능

In [5]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
#from dask import dataframe
import re
import numpy as np
import seaborn as sbn
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet

In [1]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#데이터 로드 경로 지정
import os
path = '/content/drive/MyDrive/ESAA(OB)/' # 폴더 경로
os.chdir(path) # 해당 폴더로 이동

In [7]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
valid = pd.read_csv("validation_sample.csv")

In [13]:
train.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [12]:
valid

,full_log
0,type=ANOM_PROMISCUOUS msg=audit(1600402733.466...
1,"oscap: msg: ""xccdf-result"", scan-id: ""00016007..."
2,Sep 22 10:56:19 localhost kernel: Out of memor...


In [14]:
test.head()

,id,full_log
0,1000000,"Feb 8 15:47:26 localhost kibana: {""type"":""err..."
1,1000001,"Sep 24 03:46:39 localhost kibana: {""type"":""err..."
2,1000002,type=SYSCALL msg=audit(1611888200.428:210563):...
3,1000003,"Jan 18 11:24:06 localhost kibana: {""type"":""err..."
4,1000004,type=SYSCALL msg=audit(1603081202.050:46851): ...


# EDA
### 1. 다양한 기호 및 숫자 제거
의미 없는 숫자와 기호들이 많아서 분류를 방해한다고 판단

In [18]:
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    #tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    for st in lit:
        st = " "+st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

In [19]:
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

In [20]:
train.head()

,id,level,full_log,pre_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err...",Sep localhost kibana type error timestamp tags...
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...,Feb localhost logstash INFO logstash outputs e...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp tags...
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp tags...
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...,type SYSCALL msg audit arch syscall success ye...


In [17]:
# train 데이터의 첫 번째 full_log 확인
train.full_log[0]

'Sep 24 10:02:22 localhost kibana: {"type":"error","@timestamp":"2020-09-24T01:02:22Z","tags":["warning","stats-collection"],"pid":6458,"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\\n    at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\\n    at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\\n    at process._tickCallback (internal/process/next_tick.js:61:11)"},"message":"No Living connections"}'

In [21]:
# train 데이터의 첫 번째 pre_log 확인
train.pre_log[0]

'Sep localhost kibana type error timestamp tags warning stats collection pid level error error message No Living connections name Error stack Error No Living connections at sendReqWithConnection usr share kibana node modules elasticsearch src lib transport js at next usr share kibana node modules elasticsearch src lib connection pool js at process tickCallback internal process next tick js message No Living connections '

### 2. 영어 단어가 아니거나 3글자 미만인 경우에는 삭제
Validation 7등급을 보았을 때, 의미가 있는 영어 단어들이 새로 들어오는 경향이 있다고 판단했습니다.

In [22]:
#포함되어 있는 단어 모음 생성
def count_word(data):
    tem = list(data['pre_log'].str.split(" "))
    all_word = []
    for word in tem:
        all_word.extend(word)
    words = pd.Series(all_word)
    return words.value_counts()

In [23]:
train_count = count_word(train)
test_count = count_word(test)

In [24]:
# train 데이터에서 각 단어의 빈도수
train_count

type         742245
audit        621461
error        547791
msg          502170
localhost    429812
              ...  
Bp                1
YFBjyluuP         1
BkTkqyKr          1
yaWz              1
faaabc            1
Length: 13103, dtype: int64

In [25]:
# train_count의 모든 단어들을 list 형태로 저장
train_words = list(set(train_count.index))
test_words = list(set(test_count.index))

In [29]:
#영어 단어 + 3글자 이상인 경우만 True
from nltk.corpus import words
def check_words(word_list: list):
    re = [False] * len(word_list)
    for i,word in enumerate(word_list):
        if len(word) < 3:
            continue
        word = word.lower()
        if word in words.words():
            re[i] = True
    return re

word_list = ['Promiscuous', 'ab', 'nihongo', 'abstract', 'pedo','gid']

In [31]:
#check_words는 단어인지를 확인해주고 결과를 t/f로 반환
import nltk
nltk.download('words')

check_words(word_list)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


[True, False, False, True, False, True]

In [44]:
train_tf = check_words(train_words)
test_tf = check_words(test_words)

In [46]:
#단어인지 확인해주는 딕셔너리 만들어주기 (train_isword, test_isword)
train_isword = dict()
test_isword = dict()

total_words = list(train_words)
test_words = list(test_words)

for i in range(len(train_words)):
    train_isword[train_words[i].lower()] = train_tf[i]
    
for i in range(len(test_words)):
    test_isword[test_words[i].lower()] = test_tf[i]
    

In [49]:
train_isword

{'': False,
 'fssnap': False,
 'mlocate': False,
 'previewer': False,
 'maps': False,
 'te': False,
 'efceef': False,
 'effc': False,
 'dmesg': False,
 'eceaa': False,
 'ecdb': False,
 'vve': False,
 'adfb': False,
 'gde': False,
 'bzless': False,
 'traps': True,
 'smokeping': False,
 'fcfde': False,
 'porttype': False,
 'png': False,
 'mkdkghcbyokothzhpq': False,
 'postlogin': False,
 'fu': False,
 'ln': False,
 'lslocks': False,
 'ffcea': False,
 'likewise': True,
 'oxvxst': False,
 'lsys': False,
 'counter': True,
 'ccf': False,
 'fbeef': False,
 'fbeeedb': False,
 'bebc': False,
 'cm': False,
 'epel': False,
 'lsblk': False,
 'jpg': False,
 'postgres': False,
 'ffddaa': False,
 'afdc': False,
 'clk': False,
 'faeada': False,
 'cead': False,
 'commandstopped': False,
 'immodules': False,
 'daed': False,
 'bthenum': False,
 'syntp': False,
 'pskxc': False,
 'dffe': False,
 'wzb': False,
 'fffbdf': False,
 'kwmgxcbalm': False,
 'closing': False,
 'fbdcdca': False,
 'fbbeefee': False,


### 3. Train, Test 에 각각 적용

In [47]:
import copy

def checking_train(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if train_isword[word]:
            re[i] = True
    return re


def checking_test(data):
    re = [False] * len(data)
    for i,word in enumerate(data):
        if test_isword[word]:
            re[i] = True
    return re



def cutt_train(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_train(splited)
    c = np.array(splited)
    real_words = list(c[check])    
    tem = " ".join(real_words)
    #tem = tem.lower() 
    return tem

def cutt_test(data):
    data = data.lower()
    splited = data.split(" ")
    check = checking_test(splited)
    c = np.array(splited)
    real_words = list(c[check])  
    tem = " ".join(real_words)
    #tem = tem.lower()
    return tem

In [48]:
train['cut'] = train['pre_log'].map(cutt_train)
test['cut'] = test['pre_log'].map(cutt_test)

In [50]:
del train['full_log']
del test['full_log']

del train['pre_log']
del test['pre_log']

In [51]:
train.head()

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...


train.to_csv("sub_train3.csv",index=False)
test.to_csv("sub_test3.csv",index=False)


train.to_csv("only_train.csv",index =False)
test.to_csv("only_test.csv",index =False)

### 4. 나중에 사용될 inornot 리스트

train 데이터에 100퍼센트 동일한 문장이 있는지 확인해주는 변수 입니다. 

In [52]:
train_bag = train['cut'].unique()
len(train_bag)

6544

In [54]:
#inornot을 통해서 train안에 test와 같은 문장이 있으면 0 없으면 1
from tqdm import tqdm  # 파이썬 진행률 프로세스바
inornot = np.zeros(len(test))
for i in tqdm(range(len(test))):
    tem = test.iloc[i]['cut']
    if tem not in train_bag:
        inornot[i] = 1

100%|██████████| 1418916/1418916 [14:00<00:00, 1688.47it/s]


# 모델

train = pd.read_csv("sub_train3.csv")
test= pd.read_csv("sub_test3.csv")

In [55]:
train.head(20)

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
5,5,1,type audit arch success yes exit gid suid none...
6,6,0,type error warning collection level error erro...
7,7,0,warn resurrect connection dead instance but go...
8,8,0,type error warning collection level error erro...
9,9,0,type log warning message error living share no...


In [56]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

#남은 단어가 없는 경우에는 'missing' 으로 대체해주었습니다.
train['cut'] = train['cut'].replace('','missing',regex=True)
test['cut'] = test['cut'].replace('','missing',regex=True)



train = train.fillna("missing")
test = test.fillna("missing")

train

,id,level,cut
0,0,0,type error warning collection level error erro...
1,1,0,action with response code type unavailable exc...
2,2,0,type error warning collection level error erro...
3,3,0,type error warning collection level error erro...
4,4,1,type audit arch success yes exit gid suid none...
...,...,...,...
472967,472967,0,error
472968,472968,1,type audit arch success yes exit gid suid none...
472969,472969,0,type log error task manager message poll for w...
472970,472970,0,type error warning collection level error erro...


In [57]:
train_text = list(train['cut'])
train_level = np.array(train['level'])

test_text = list(test['cut'])

In [60]:
vectorizer=CountVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [61]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [63]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [66]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.5) & (results == 1))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 2))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.95) & (results == 3))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 4))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 5))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.58) & (results == 6))[0]]=7

In [67]:
#Validation을 통한 threshold 조절
results[np.where((np.max(results_proba, axis=1)<0.94001) & (np.max(results_proba, axis=1) > 0.93999) & (results == 1))[0]]=7 
results[np.where((np.max(results_proba, axis=1)<0.611657) & (np.max(results_proba, axis=1) > 0.6116568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.571657) & (np.max(results_proba, axis=1) > 0.5716568) & (results == 0))[0]]=7
results[np.where((np.max(results_proba, axis=1)<0.68001) & (np.max(results_proba, axis=1) > 0.67999) & (results == 5))[0]]=7 

# 결과 저장

In [71]:
submission=pd.read_csv('sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

0    1002369
1     396443
3      12813
5       6418
7        780
2         34
4         34
6         25
Name: level, dtype: int64

In [72]:
submission

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [ ]:
submission.to_csv("tem_answer.csv",index = False)

# 추가작업
TEST 데이터에 동일한 문장이 있는지를 생성했던 컬럼을 추가해주는 작업입니다.

In [73]:
submission['inornot'] = pd.Series(inornot)

In [74]:
cal = submission[(submission['level']==7) & (submission['inornot'] == 0)].index
not7_id = list(test.iloc[cal]['id'])
pd.Series(not7_id).to_csv("not7_id.csv",index= False)

# 추가 작업

위의 방식으로 많이 전처리를 하면 7을 걸러내기 위한 threshold 를 잡기는 유리하지만, 지도 학습 자체의 성능은 떨어지는 것을 확인 하였습니다. 따라서, 7를 제외한 나머지 등급에는 전처리가 덜 진행된 Data로 지도학습 분류를 하였습니다. TfidfVectorizer이 더 성능이 우수하여 사용했습니다. 

In [76]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [77]:
train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [78]:
train = train.fillna("missing")
test = test.fillna("missing")

In [79]:
train_text=list(train['full_log'])
train_level=np.array(train['level'])

test_text=list(test['full_log'])
#ids=list(test['id'])

In [80]:
vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [81]:
forest=RandomForestClassifier(n_estimators=100,random_state = 1 )
forest.fit(train_features,train_level)

RandomForestClassifier(random_state=1)

In [82]:
results=forest.predict(test_features)
results_proba=forest.predict_proba(test_features)

In [83]:
submission=pd.read_csv('sample_submission.csv')
submission['level']=results
submission['level'].value_counts()

0    1002770
1     396532
3      12997
5       6524
2         34
4         34
6         25
Name: level, dtype: int64

In [84]:
not7_id = pd.read_csv('not7_id.csv')
not7_id = list(not7_id['0'])

In [ ]:
#위에서 잡은 7 합치기 & train에 있는 경우는 사용하지 않기
only = pd.read_csv("tem_answer.csv")
le7 = list(only[only['level']==7]['id'])
le7_update = list((set(le7))-set(not7_id))
print(len(le7))
print(len(le7_update))
idx = submission[submission['id'].isin(le7_update)].index
submission.iloc[idx] = 7
submission['level'].value_counts()

780
581


0    1002512
1     396348
3      12915
5       6467
7        581
4         34
2         34
6         25
Name: level, dtype: int64

In [ ]:
who = submission.level
submission=pd.read_csv('sample_submission.csv')
submission['level'] = who
submission['level'].value_counts()

0    1002512
1     396348
3      12915
5       6467
7        581
4         34
2         34
6         25
Name: level, dtype: int64

In [ ]:
submission.to_csv("581_final_answer.csv",index = False)